In [ ]:
!pip install anytree

In [ ]:
import pandas as pd
from anytree import Node, RenderTree, AsciiStyle, PreOrderIter
from copy import deepcopy

In [ ]:
# Level of the cut
X = 2

# Read CSV
df = pd.read_csv('FinalDataset/DBpediaClasses.csv')

In [ ]:
# Create dictionary to store nodes
nodes = {}

# Build the tree and the nodes
for _, row in df.iterrows():
    parent_name = row['SubClass']
    child_name = row['Class']
    
    # Creat parent node if it does not exist
    if parent_name not in nodes:
        nodes[parent_name] = Node(parent_name)
    
    # Create children node and assign it to the parent
    if child_name not in nodes:
        nodes[child_name] = Node(child_name, parent=nodes[parent_name])
    else:
        nodes[child_name].parent = nodes[parent_name]

root_nodes = [node for node in nodes.values() if node.is_root]

In [ ]:
# Show the tree
for root_node in root_nodes:
    for pre, fill, node in RenderTree(root_node, style=AsciiStyle()):
        print(f"{pre}{node.name}")

In [ ]:
root_nodes_copy = deepcopy(root_nodes)

In [ ]:
def prune_tree(node, max_depth, current_depth=1):
    if current_depth >= max_depth:
        node.children = []
    else:
        for child in node.children:
            prune_tree(child, max_depth, current_depth + 1)

# Cut the tree
for root in root_nodes_copy:
    prune_tree(root, X)

# Show the new tree
for pre, fill, node in RenderTree(root_nodes_copy[0]):
    print("%s%s" % (pre, node.name))

In [ ]:
df_new = pd.read_csv('FinalDataset/final_dataset.csv')

In [ ]:
def get_new_class(node, podado_root):
    current_node = node
    while current_node and not any(p.name == current_node.name for p in podado_root.descendants):
        current_node = current_node.parent
    return current_node.name if current_node else None

nodes = {node.name: node for node in root_nodes[0].descendants}
nodes[root.name] = root

df_new['NewClass'] = df_new['Class'].apply(lambda x: get_new_class(nodes[x], root_nodes_copy[0]))
num_labels = num_unique_labels = df_new['NewClass'].nunique()

In [ ]:
print(df_new)

In [ ]:
df_new.to_csv('FinalDataset/polished_dataset_'+str(num_labels)+'.csv', index=False)